In [40]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import time 
from tensorflow.keras.models import Sequential, load_model
from keras.layers import SimpleRNN, Dense
import tensorflow as tf
import keras
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [41]:
data_path_and_labels = pd.read_csv("../mapped_data/xalan-2.4.csv")
data_path_and_labels['data_path'][0]

'dataset/promise/defect data/xalan-2.4/xalan-j_2_4_0/src/org/apache/xalan/Version.java.restrictedcontent.embed'

In [42]:
data_path_and_labels['data_path'] = '../' + data_path_and_labels['data_path']
data_path_and_labels['data_path'][0]

'../dataset/promise/defect data/xalan-2.4/xalan-j_2_4_0/src/org/apache/xalan/Version.java.restrictedcontent.embed'

In [43]:
data_path_and_labels['data_path'] = data_path_and_labels['data_path'].str.replace('.embed','')
data_path_and_labels['data_path'][0]

'../dataset/promise/defect data/xalan-2.4/xalan-j_2_4_0/src/org/apache/xalan/Version.java.restrictedcontent'

In [44]:
words = []
for index, row in data_path_and_labels.iterrows():
    with open(row['data_path']) as f:
        lines = f.read().splitlines()
    words.append(lines)

In [45]:
d = pd.read_csv("../mapped_data/xalan-2.4.csv")
d['data_path'][0]

'dataset/promise/defect data/xalan-2.4/xalan-j_2_4_0/src/org/apache/xalan/Version.java.restrictedcontent.embed'

In [46]:
d['data_path'] = '../' + d['data_path']
d['data_path'][0]

'../dataset/promise/defect data/xalan-2.4/xalan-j_2_4_0/src/org/apache/xalan/Version.java.restrictedcontent.embed'

In [47]:
d

,data_path,bugs
0,../dataset/promise/defect data/xalan-2.4/xalan...,0
1,../dataset/promise/defect data/xalan-2.4/xalan...,2
2,../dataset/promise/defect data/xalan-2.4/xalan...,0
3,../dataset/promise/defect data/xalan-2.4/xalan...,0
4,../dataset/promise/defect data/xalan-2.4/xalan...,0
...,...,...
671,../dataset/promise/defect data/xalan-2.4/xalan...,0
672,../dataset/promise/defect data/xalan-2.4/xalan...,0
673,../dataset/promise/defect data/xalan-2.4/xalan...,0
674,../dataset/promise/defect data/xalan-2.4/xalan...,0


In [48]:
train, test = train_test_split(d, test_size=0.2)

In [49]:
test_x = []
for index, row in test.iterrows():
    with open(row['data_path']) as f:
        lines = list(map(float, f.read().splitlines())) 
    test_x.append(lines)

In [50]:
train_x = []
for index, row in train.iterrows():
    with open(row['data_path']) as f:
        lines = list(map(float, f.read().splitlines())) 
    train_x.append(lines)

In [51]:
vocab = float(max(np.max(train_x), np.max(test_x)))
vocab

9459.0

In [52]:
train_y = train[' bugs'].to_numpy()
test_y = test[' bugs'].to_numpy()

In [53]:
train_y

array([0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 3, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3,
       0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 2, 1,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 1, 0, 0,
       1, 0, 0, 0, 2, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 2, 0, 0, 1, 0, 0, 2, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 2, 0, 0, 0, 1, 0, 0, 1, 0, 0,
       1, 0, 2, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 0, 0, 0, 1,

In [54]:
train_x = np.array(train_x)
test_x = np.array(test_x)

In [58]:
train_x.shape

(540, 7367)

In [60]:
# Designing and initializing the model.
from keras.layers import LSTM, Dense, SimpleRNN
model = Sequential()
model.add(tf.keras.layers.Embedding(10671, 30, input_length=train_x.shape[1]))
#model.add(tf.keras.layers.Flatten())
model.add(LSTM(200, return_sequences=True))
model.add(LSTM(100, return_sequences=True))
model.add(LSTM(50, dropout = 0.2, return_sequences=True))
model.add(LSTM(20, return_sequences=False))
model.add(Dense(1, activation = 'relu'))
model.compile(loss = 'mse' , optimizer = keras.optimizers.Adam(learning_rate=0.0001) , metrics = ['mse', 'mae', tf.keras.metrics.RootMeanSquaredError(), tf.keras.metrics.MeanSquaredLogarithmicError()] )

In [61]:
# Fitting the model on training data.
import datetime
log_dir = '/home/bavanya/Desktop/tensorboard/' + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
history = model.fit(train_x, train_y, epochs = 100, callbacks=[tensorboard_callback])

Epoch 1/100
 9/17 [==============>...............] - ETA: 3:23 - loss: 0.5667 - mse: 0.5667 - mae: 0.3086 - root_mean_squared_error: 0.7528 - mean_squared_logarithmic_error: 0.1372

KeyboardInterrupt: 

In [19]:
%tensorboard --logdir '/home/bavanya/Desktop/tensorboard/'

Reusing TensorBoard on port 6006 (pid 21467), started 0:41:07 ago. (Use '!kill 21467' to kill it.)

In [20]:
score = model.evaluate(test_x, test_y)
score

5/5 [==============================] - 0s 6ms/step - loss: 0.7426 - mse: 0.7426 - mae: 0.2426 - root_mean_squared_error: 0.8618 - mean_squared_logarithmic_error: 0.1393


[0.7426470518112183,
 0.7426470518112183,
 0.24264705181121826,
 0.8617697358131409,
 0.13933055102825165]

In [18]:
data_path_and_labels = pd.read_csv("../mapped_data/xalan-2.4.csv")
data_path_and_labels['data_path'][0]

'dataset/promise/defect data/xalan-2.4/xalan-j_2_4_0/src/org/apache/xalan/Version.java.restrictedcontent.embed'

In [19]:
data_path_and_labels['data_path'] = '../' + data_path_and_labels['data_path']
data_path_and_labels['data_path'][0]

'../dataset/promise/defect data/xalan-2.4/xalan-j_2_4_0/src/org/apache/xalan/Version.java.restrictedcontent.embed'

In [20]:
embeddings = []
for index, row in data_path_and_labels.iterrows():
    with open(row['data_path']) as f:
        lines = list(map(float, f.read().splitlines())) 
    embeddings.append(lines)

In [21]:
df = pd.DataFrame(embeddings)
df

,0,1,2,3,4,5,6,7,8,9,...,7357,7358,7359,7360,7361,7362,7363,7364,7365,7366
0,2456.0,5449.0,2250.0,8094.0,5261.0,5004.0,5094.0,5281.0,4878.0,4878.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,994.0,5463.0,5756.0,8094.0,517.0,5175.0,2250.0,8094.0,519.0,4397.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1254.0,4397.0,2562.0,9316.0,2569.0,1020.0,1020.0,5139.0,2270.0,8663.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2448.0,2455.0,2455.0,8051.0,8019.0,5441.0,5756.0,8094.0,8599.0,4142.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1990.0,2455.0,6742.0,2455.0,6287.0,3529.0,6589.0,79.0,79.0,5756.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
671,4808.0,1806.0,8094.0,3805.0,1806.0,3805.0,3611.0,3611.0,5756.0,6167.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
672,4808.0,1806.0,8094.0,3805.0,1806.0,3805.0,3611.0,3611.0,5756.0,3611.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
673,4808.0,1806.0,8094.0,3805.0,1806.0,3805.0,3611.0,3611.0,5756.0,6167.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
674,4808.0,1806.0,8094.0,3805.0,1806.0,3805.0,3611.0,3611.0,5756.0,6167.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [22]:
data = pd.concat([data_path_and_labels, df], axis = 1)
data = data.drop('data_path', 1)
data

,bugs,0,1,2,3,4,5,6,7,8,...,7357,7358,7359,7360,7361,7362,7363,7364,7365,7366
0,0,2456.0,5449.0,2250.0,8094.0,5261.0,5004.0,5094.0,5281.0,4878.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,994.0,5463.0,5756.0,8094.0,517.0,5175.0,2250.0,8094.0,519.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0,1254.0,4397.0,2562.0,9316.0,2569.0,1020.0,1020.0,5139.0,2270.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0,2448.0,2455.0,2455.0,8051.0,8019.0,5441.0,5756.0,8094.0,8599.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0,1990.0,2455.0,6742.0,2455.0,6287.0,3529.0,6589.0,79.0,79.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
671,0,4808.0,1806.0,8094.0,3805.0,1806.0,3805.0,3611.0,3611.0,5756.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
672,0,4808.0,1806.0,8094.0,3805.0,1806.0,3805.0,3611.0,3611.0,5756.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
673,0,4808.0,1806.0,8094.0,3805.0,1806.0,3805.0,3611.0,3611.0,5756.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
674,0,4808.0,1806.0,8094.0,3805.0,1806.0,3805.0,3611.0,3611.0,5756.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [23]:
train, test = train_test_split(data, test_size=0.2)

In [24]:
train

,bugs,0,1,2,3,4,5,6,7,8,...,7357,7358,7359,7360,7361,7362,7363,7364,7365,7366
555,1,2204.0,2204.0,3771.0,309.0,7513.0,5756.0,3305.0,5756.0,3771.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
31,0,1255.0,192.0,1401.0,192.0,4397.0,2562.0,9316.0,2569.0,2377.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
374,0,1912.0,7823.0,8050.0,1997.0,7833.0,5353.0,7833.0,5353.0,7833.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
206,0,278.0,1063.0,2903.0,1609.0,1609.0,7911.0,192.0,192.0,7911.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
575,0,448.0,8575.0,1098.0,8575.0,1102.0,8575.0,1103.0,4847.0,433.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
482,0,4397.0,2562.0,9316.0,2569.0,2250.0,7167.0,4397.0,8739.0,9316.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
388,0,280.0,189.0,6852.0,189.0,1319.0,1319.0,2250.0,1313.0,2250.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
176,0,2019.0,1876.0,2250.0,6068.0,2250.0,8106.0,1876.0,2250.0,2250.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
424,0,1745.0,2569.0,6476.0,2455.0,6610.0,1745.0,7572.0,2569.0,6476.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [25]:
test

,bugs,0,1,2,3,4,5,6,7,8,...,7357,7358,7359,7360,7361,7362,7363,7364,7365,7366
179,0,2434.0,3154.0,7646.0,7658.0,1886.0,2250.0,9141.0,4730.0,5610.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
535,0,190.0,7520.0,2562.0,8106.0,2562.0,5610.0,226.0,5419.0,8094.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
37,0,1016.0,70.0,6668.0,8415.0,9150.0,70.0,6668.0,9150.0,5129.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11,1,1026.0,1026.0,5756.0,6568.0,5756.0,6520.0,8361.0,5675.0,5756.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
83,1,1082.0,1319.0,1319.0,2270.0,6854.0,1082.0,8662.0,2270.0,6854.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64,0,3806.0,2593.0,4223.0,1006.0,9064.0,2034.0,8172.0,2034.0,7237.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
664,0,4808.0,1806.0,8094.0,3805.0,1806.0,3805.0,3611.0,3611.0,5756.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
615,0,2256.0,5756.0,6320.0,2250.0,6643.0,5756.0,6502.0,5756.0,6645.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
198,0,2236.0,1063.0,2674.0,1063.0,3128.0,2236.0,4609.0,1938.0,3404.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [26]:
Y_train = train[' bugs']
Y_test = test[' bugs']

In [27]:
train_y = Y_train.to_numpy()
test_y = Y_test.to_numpy()

In [28]:
train = train.drop(' bugs', axis=1)
test = test.drop(' bugs', axis=1)

In [29]:
X_train = train.values
X_test = test.values

In [30]:
train_x = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))
test_x = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))

In [37]:
# Designing and initializing the model.
from keras.layers import LSTM, Dense, SimpleRNN
# Designing and initializing the model.
model = Sequential()
model.add(LSTM(200, return_sequences=True))
model.add(LSTM(100, return_sequences=True))
model.add(LSTM(50, dropout = 0.2, return_sequences=True))
model.add(LSTM(20, return_sequences=False))
model.add(Dense(1, activation = 'relu'))
model.compile(loss = 'mse' , optimizer = keras.optimizers.Adam(learning_rate=0.0001) , metrics = ['mse', 'mae', tf.keras.metrics.RootMeanSquaredError(), tf.keras.metrics.MeanSquaredLogarithmicError()] )

In [38]:
# Fitting the model on training data.
history = model.fit(train_x, train_y, epochs = 100)

Epoch 1/100
17/17 [==============================] - 4s 33ms/step - loss: 0.4761 - mse: 0.4761 - mae: 0.2506 - root_mean_squared_error: 0.6900 - mean_squared_logarithmic_error: 0.1333
Epoch 2/100
17/17 [==============================] - 1s 34ms/step - loss: 0.4711 - mse: 0.4711 - mae: 0.2546 - root_mean_squared_error: 0.6864 - mean_squared_logarithmic_error: 0.1305
Epoch 3/100
17/17 [==============================] - 1s 34ms/step - loss: 0.4663 - mse: 0.4663 - mae: 0.2588 - root_mean_squared_error: 0.6829 - mean_squared_logarithmic_error: 0.1278
Epoch 4/100
17/17 [==============================] - 1s 34ms/step - loss: 0.4614 - mse: 0.4614 - mae: 0.2628 - root_mean_squared_error: 0.6793 - mean_squared_logarithmic_error: 0.1252
Epoch 5/100
17/17 [==============================] - 1s 38ms/step - loss: 0.4568 - mse: 0.4568 - mae: 0.2671 - root_mean_squared_error: 0.6759 - mean_squared_logarithmic_error: 0.1228
Epoch 6/100
17/17 [==============================] - 1s 34ms/step - loss: 0.4501

17/17 [==============================] - 1s 35ms/step - loss: 0.2930 - mse: 0.2930 - mae: 0.2881 - root_mean_squared_error: 0.5413 - mean_squared_logarithmic_error: 0.0816
Epoch 46/100
17/17 [==============================] - 1s 34ms/step - loss: 0.2856 - mse: 0.2856 - mae: 0.2877 - root_mean_squared_error: 0.5345 - mean_squared_logarithmic_error: 0.0801
Epoch 47/100
17/17 [==============================] - 1s 35ms/step - loss: 0.2984 - mse: 0.2984 - mae: 0.2691 - root_mean_squared_error: 0.5463 - mean_squared_logarithmic_error: 0.0778
Epoch 48/100
17/17 [==============================] - 1s 33ms/step - loss: 0.2854 - mse: 0.2854 - mae: 0.2686 - root_mean_squared_error: 0.5342 - mean_squared_logarithmic_error: 0.0762
Epoch 49/100
17/17 [==============================] - 1s 34ms/step - loss: 0.2834 - mse: 0.2834 - mae: 0.2761 - root_mean_squared_error: 0.5324 - mean_squared_logarithmic_error: 0.0780
Epoch 50/100
17/17 [==============================] - 1s 36ms/step - loss: 0.2850 - mse:

17/17 [==============================] - 1s 33ms/step - loss: 0.1918 - mse: 0.1918 - mae: 0.2267 - root_mean_squared_error: 0.4380 - mean_squared_logarithmic_error: 0.0596
Epoch 90/100
17/17 [==============================] - 1s 34ms/step - loss: 0.1943 - mse: 0.1943 - mae: 0.2318 - root_mean_squared_error: 0.4408 - mean_squared_logarithmic_error: 0.0602
Epoch 91/100
17/17 [==============================] - 1s 36ms/step - loss: 0.1866 - mse: 0.1866 - mae: 0.2243 - root_mean_squared_error: 0.4319 - mean_squared_logarithmic_error: 0.0579
Epoch 92/100
17/17 [==============================] - 1s 34ms/step - loss: 0.1840 - mse: 0.1840 - mae: 0.2184 - root_mean_squared_error: 0.4289 - mean_squared_logarithmic_error: 0.0558
Epoch 93/100
17/17 [==============================] - 1s 34ms/step - loss: 0.1842 - mse: 0.1842 - mae: 0.2163 - root_mean_squared_error: 0.4292 - mean_squared_logarithmic_error: 0.0566
Epoch 94/100
17/17 [==============================] - 1s 33ms/step - loss: 0.1824 - mse:

In [39]:
score = model.evaluate(test_x, test_y)
score

5/5 [==============================] - 1s 6ms/step - loss: 0.2754 - mse: 0.2754 - mae: 0.2726 - root_mean_squared_error: 0.5248 - mean_squared_logarithmic_error: 0.0914


[0.2754025459289551,
 0.2754025459289551,
 0.27256566286087036,
 0.5247880816459656,
 0.09137754887342453]